In [1]:
import pandas as pd
import numpy as np
import torch
from typing import List
from torch_geometric.data import Dataset
from torch_geometric.data import Data
import torch.nn.functional as F

class ValidTimeSeriesDataset(Dataset):
    def __init__(self, df: pd.DataFrame, edge_index: List, window_size: int):
        super(ValidTimeSeriesDataset, self).__init__(None, None, None)
        self.edge_index = edge_index
        self.window_size = window_size
        self._read_data(df)
        self._get_edges()
        self._set_targets_and_features()

    def _read_data(self, df: pd.DataFrame):
        self.df = df
        self.pump_names = df.filter(regex='^P\\d+$').columns.tolist()
        self.flags_names = df.filter(regex=r'^P\d+_flag$').columns.tolist()
        self.flux_names = df.filter(regex='^Q\\d+$').columns.tolist()

        self.accident_labels = df['anomaly'].values.astype(np.int8)
        self.press_values = torch.from_numpy(df[self.pump_names].values.astype(np.float32)) # (T, p_count)
        self.flux_values = torch.from_numpy(df[self.flux_names].values.astype(np.float32)) # (T, f_count)
        self.labels = torch.from_numpy(df[self.flags_names].values.astype(np.float32)) # (T, p_count)

        potential_starts = torch.arange(0, len(self.df) - self.window_size)
        self.valid_starts = [
            idx for idx in potential_starts 
            if np.all(self.accident_labels[idx:idx+self.window_size] == 0)
        ]

    def _get_edges(self):
        self._edges = torch.tensor(self.edge_index, dtype=torch.long).T
    
    def len(self):
        return len(self.valid_starts)
    
    def _normalize_window(self, window: torch.Tensor):
        # window => (w, n) => (n, w)
        window = window.T
        min_val = window.min(dim=1, keepdim=True)[0]
        max_val = window.max(dim=1, keepdim=True)[0]
        # max_val - min_val이 0인 경우를 방지하기 위해 1e-10을 더함
        normalized_x = (window - min_val) / (max_val - min_val + 1e-10)
        return normalized_x
    
    def _set_targets_and_features(self):
        valid_press_values = torch.stack([self._normalize_window(self.press_values[idx:idx+self.window_size]) for idx in self.valid_starts]) # (t, p_count, w)
        valid_flux_values = torch.stack([self._normalize_window(self.flux_values[idx:idx+self.window_size]) for idx in self.valid_starts]) # (t, f_count, w)
        self.features = valid_press_values
    
    def get(self, i):
        data = Data(x=self.features[i], edge_index=self._edges)
        return data


class TimeSeriesDataset(Dataset):
    def __init__(self, df: pd.DataFrame, edge_index: List, window_size: int):
        super(TimeSeriesDataset, self).__init__(None, None, None)
        self.edge_index = edge_index
        self.window_size = window_size
        self._read_data(df)
        self._get_edges()

    def _read_data(self, df: pd.DataFrame):
        self.df = df
        self.pump_names = df.filter(regex='^P\\d+$').columns.tolist()
        self.flags_names = df.filter(regex=r'^P\d+_flag$').columns.tolist()
        self.flux_names = df.filter(regex='^Q\\d+$').columns.tolist()

        self.accident_labels = df['anomaly'].values.astype(np.int8)
        self.press_values = torch.from_numpy(df[self.pump_names].values.astype(np.float32)) # (T, p_count)
        self.flux_values = torch.from_numpy(df[self.flux_names].values.astype(np.float32)) # (T, f_count)
        self.labels = torch.from_numpy(df[self.flags_names].values.astype(np.float32)) # (T, p_count)

    def _get_edges(self):
        self._edges = torch.tensor(self.edge_index, dtype=torch.long).T
    
    def len(self):
        return len(self.press_values) - self.window_size
    
    def _normalize_window(self, window: torch.Tensor):
        # window => (w, n) => (n, w)
        window = window.T
        mins = torch.min(window, dim=1, keepdim=True).values
        maxs = torch.max(window, dim=1, keepdim=True).values
        normalized_window = (window - mins) / (maxs - mins)
        return normalized_window
      
    
    def get(self, i):
        window = self._normalize_window(self.press_values[i:i+self.window_size]) # (n, w)
        data = Data(x=window, edge_index=self._edges, y=self.labels[i:i+self.window_size])
        return data



In [2]:
edge_index_A = [
    (0, 4), (0, 5), (0, 7), (0, 8), (0, 6), (0, 11), (0, 9), (0, 18), (0, 25),
    (1, 3),
    (2, 4), (2, 5), (2, 7), (2, 8), (2, 6), (2, 11), (2, 9), (2, 18), (2, 25),
    (3, 4), (3, 5), (3, 7), (3, 8), (3, 6), (3, 11), (3, 9), (3, 18), (3, 25),
    (4, 7), (4, 8), (4, 6), (4, 11), (4, 9), (4, 18), (4, 25),
    (5, 7), (5, 8), (5, 6), (5, 11), (5, 9), (5, 18), (5, 25),
    (6, 8),
    (7, 11), (7, 9), (7, 18), (7, 25),
    (8, 11), (8, 9), (8, 18), (8, 25),
    (9, 10),
    (10, 13),
    (11, 12), (11, 17),
    (12, 13), (12, 17),
    (13, 12), (13, 14),
    (14, 15),
    (15, 16), (15, 18),
    (17, 19), (17, 18),
    (18, 17), (18, 21), (18, 25),
    (19, 20),
    (20, 23),
    (21, 22),
    (22, 24)
]

window_size = 60*24*7 # 10080
dataframe = pd.read_csv('/home/wujin/workspace/competition2/datasets/train/TRAIN_A.csv')
dataset = ValidTimeSeriesDataset(df=dataframe, edge_index=edge_index_A, window_size=window_size)
test_dataset = TimeSeriesDataset(df=dataframe, edge_index=edge_index_A, window_size=window_size)

In [4]:
print(dataset[1].x.shape)

torch.Size([26, 10080])


In [5]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

train_size = int(len(dataset) * 0.8)  # 80%를 훈련 데이터로
val_size = len(dataset) - train_size  # 나머지 20%를 검증 데이터로

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
from tqdm import tqdm
from models.model import GraphSAGEVAE
from torch_geometric.utils import negative_sampling

%load_ext autoreload
%autoreload 2

device = torch.device('cuda:0')
model = GraphSAGEVAE(in_channels=window_size, hidden_channels=512, out_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 30
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0.0
    with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{n_epochs}", unit="batch") as t:
        for batch in t:
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            loss, recon_loss, kl_loss = model(x, edge_index)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            t.set_postfix(loss=loss.item())

    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{n_epochs}, Average Train Loss: {avg_epoch_loss:.8f}")

    model.eval()
    eval_loss = 0.0
    for batch in tqdm(val_loader, desc="Eval"):
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            loss, recon_loss, kl_loss = model(x, edge_index)
            eval_loss += loss.item()
    
    avg_eval_loss = eval_loss / len(val_loader)
    print(f"Epoch {epoch + 1}/{n_epochs}, Average Eval Loss: {avg_eval_loss:.8f}")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Epoch 1/30: 100%|██████████| 425/425 [00:38<00:00, 11.01batch/s, loss=0.707]


Epoch 1/30, Average Train Loss: 1.67423802


Eval: 100%|██████████| 107/107 [00:03<00:00, 28.27it/s]


Epoch 1/30, Average Eval Loss: 0.69580389


...

Epoch 11/30, Average Train Loss: 0.69415701


Eval: 100%|██████████| 107/107 [00:03<00:00, 28.21it/s]


Epoch 11/30, Average Eval Loss: 0.69326186


Epoch 12/30:  93%|█████████▎| 395/425 [00:36<00:02, 10.97batch/s, loss=0.694]


KeyboardInterrupt: 

In [9]:
from torch_geometric.utils import to_dense_adj

threshold_percentile = 95
total_matched_abnormal = 0
total_false_positives = 0
total_pred_abnormal = 0
total_gt_abnormal = 0

model.eval()
for batch in tqdm(test_loader, desc="Test"):
        x = batch.x.to(device)
        edge_index = batch.edge_index.to(device)
        z, mean, log_std = model(x, edge_index, neg_edge_index)
        adj = to_dense_adj(edge_index, max_num_nodes=batch.num_nodes).squeeze(0)
        with torch.no_grad():
            # 모든 엣지 쌍에 대해 내적 계산
            # z: [N, d], z @ z.T: [N, N]
            pred_adj = torch.sigmoid(torch.matmul(z, z.t()))
        
        # 노드 수준의 재구성 오류 계산
        # 각 노드 i에 대해 실제 인접 벡터 adj[i], 예측 pred_adj[i] 비교
        # 여기서는 Binary Cross Entropy 기반으로 계산 가능
        print(batch.num_nodes)
        node_scores = []
        for i in range(batch.num_nodes):
            # 실제 값 (adj의 i번째 row): 0 또는 1
            # 예측 값 (pred_adj의 i번째 row): 0과 1 사이의 확률값
            # BCE = - [y*log(p) + (1-y)*log(1-p)] 의 평균
            y = adj[i]
            p = pred_adj[i]

            # 노드 i에 연결된 엣지만 고려할 수도 있고(양성 중심),
            # 여기서는 모든 노드 쌍을 동일 비중으로 사용
            bce = -(y * torch.log(p + 1e-15) + (1 - y)*torch.log(1 - p + 1e-15))
            node_score = bce.mean().item()
            node_scores.append(node_score)

        node_scores = torch.tensor(node_scores)
        threshold = torch.quantile(node_scores, threshold_percentile / 100.0).item()
        
        # 이상치로 간주되는 노드 식별
        labels = batch.y
        anomaly_labels = (node_scores > threshold).int()
        total_matched_abnormal += ((labels == 1) & (anomaly_labels == 1)).sum().item()
        total_false_positives += ((labels == 0) & (anomaly_labels == 1)).sum().item()
        total_pred_abnormal += (anomaly_labels == 1).sum().item()
        total_gt_abnormal += (labels == 1).sum().item()

precision = total_matched_abnormal / (total_pred_abnormal + total_false_positives)
recall = total_matched_abnormal / total_gt_abnormal
f1_score = (2 * precision * recall) / (precision + recall)
print("f1 score: ", f1_score)

        

Test:   0%|          | 0/532 [00:00<?, ?it/s]

645120 1664


RuntimeError: The size of tensor a (26) must match the size of tensor b (1664) at non-singleton dimension 1

In [18]:
a = torch.Tensor([1, 0, 0, 1, 1]) # 실제
b = torch.Tensor([1, 1, 0, 0, 1]) # 예측

matched_abnormal = ((a == 1) & (b == 1)).sum().item()
false_positives = ((a == 0) & (b == 1)).sum().item()
pred_abnormal = (b == 1).sum().item()
gt_abnormal = (a == 1).sum().item()
print(gt_abnormal)

3
